In [26]:
!python -m spacy download en_core_web_lg
import os

import spacy
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [13]:
train_data = pd.read_csv(f"data/train.csv")
test_data = pd.read_csv(f"data/test.csv")

In [41]:
nlp = spacy.load('en_core_web_lg')
with nlp.disable_pipes():
    x_test = np.array([nlp(text).vector for text in tqdm(test_data.excerpt)])
with nlp.disable_pipes():
    x_train = np.array([nlp(text).vector for text in tqdm(train_data.excerpt)])

In [42]:
y = train_data["target"].values

In [43]:
NFOLDS = 5
skf = KFold(n_splits=NFOLDS)
folds = list(skf.split(x_train))

In [44]:
oof = np.zeros(y.shape)
pred = np.zeros((x_test.shape[0],))
for idx in range(NFOLDS):
    print("FOLD: ", idx + 1)
    tr_idx, val_idx = folds[idx]
    reg = LinearRegression()
    reg.fit(x_train[tr_idx], y[tr_idx])
    oof[val_idx] =reg.predict(x_train[val_idx])
    pred += reg.predict(x_test) / NFOLDS

In [45]:
mse = mean_squared_error(y, oof, squared=False)
print("OOF MSE:",mse)